# Импорт необходимых библиотек

In [93]:
import os
import warnings

from bs4 import BeautifulSoup

import requests
import urllib3
from urllib.request import urlopen
from urllib.parse import urlparse

import pandas as pd
import numpy as np

from collections import deque

from urllib.parse import urljoin

import sqlite3


warnings.filterwarnings("ignore")
urllib3.disable_warnings()

# Извлечение чистого текста из html-кода с сохранением его структуры (разметка по абзацам и заголовкам)

In [38]:
def mark_text(html_code : str):

    soup = BeautifulSoup(html_code, 'html.parser')

    headings = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
    paragraphs = ['p', 'pre']

    HEADING_MARK = '@'
    PARAGRAPH_MARK = '&'

    def process_tag(tag):
        if tag.name in headings:
            return HEADING_MARK + tag.text.strip() + '\n'
        elif tag.name in paragraphs:
            return PARAGRAPH_MARK + tag.text.strip() + '\n'
        else:
            return ''


    result = ''

    for tag in soup.find_all():
        result += process_tag(tag)

    return result

In [39]:
sample_html_code = '''
<html>
  <body>
    <h1>This is a heading</h1>
    <p>This is a paragraph.</p>
    <p>Another paragraph.</p>
  </body>
</html>
'''

print(mark_text(sample_html_code))

@This is a heading
&This is a paragraph.
&Another paragraph.



# Выгрузка из таблицы агенства RAEX необходимой информации о компаниях (названия и url)

In [40]:
pd.set_option('display.max_rows', None)

In [45]:
RAEX_PATH = 'RAEX_list.xlsx'

raex_list = pd.read_excel(RAEX_PATH)
raex_list

,№,Название,Код MOEX,Подотрасль,ESG-рейтинг,E Rank,E-рейтинг,S Rank,S-рейтинг,G Rank,G-рейтинг,Год последней оцененной отчетности,url,url_sustainability
0,1,НЛМК,NLMK,Чёрная металлургия,AA,2,AA,2,AA,21,A,2021,https://nlmk.com/ru/,https://nlmk.com/ru/sustainability/
1,2,«Полюс»,PLZL,Драгоценные металлы,AA,1,AAA,14,A,27,A,2021,https://polyus.com/ru/,https://sustainability.polyus.com/ru/
2,3,«Уралкалий»,-,Агрохимикаты,A,6,BBB,1,AA,6,AA,2021,https://www.uralkali.com/ru/,https://www.uralkali.com/ru/sustainability/
3,4,«ЭЛ5-Энерго»,ELFV,Электроэнергетика,A,9,BBB,9,A,2,AAA,2021,https://www.el5-energo.ru,https://www.el5-energo.ru/sustainability/
4,5,«Полиметалл»,POLY,Драгоценные металлы,A,4,A,5,A,12,AA,2021,https://www.polymetalinternational.com/ru/,https://www.polymetalinternational.com/ru/sust...
5,6,"«ФосАгро», группа",PHOR,Агрохимикаты,A,8,BBB,11,A,5,AA,2021,https://www.phosagro.ru,https://www.phosagro.ru/sustainability/
6,7,«МОСКОВСКИЙ КРЕДИТНЫЙ БАНК»,CBOM,Банки,A,5,A,15,A,8,AA,2021,https://mkb.ru,https://ir.mkb.ru/sustainability
7,8,"«Норильский никель», горно-металлургическая ко...",GMKN,Добыча прочих полезных ископаемых,A,12,BBB,18,A,4,AA,2021,https://www.nornickel.ru,https://www.nornickel.ru/sustainability/
8,9,«Северсталь»,CHMF,Чёрная металлургия,A,7,BBB,10,A,19,A,2021,https://severstal.com/rus/,https://severstal.com/rus/sustainable-developm...
9,10,«ЛУКОЙЛ»,LKOH,Интегрированные нефтегазовые компании,A,11,BBB,13,A,11,AA,2021,https://lukoil.ru,https://lukoil.ru/Sustainability


In [46]:
choose_url_list = raex_list.loc[raex_list['url'].notnull(), ['Название', 'url', 'url_sustainability']]
choose_url_list

,Название,url,url_sustainability
0,НЛМК,https://nlmk.com/ru/,https://nlmk.com/ru/sustainability/
1,«Полюс»,https://polyus.com/ru/,https://sustainability.polyus.com/ru/
2,«Уралкалий»,https://www.uralkali.com/ru/,https://www.uralkali.com/ru/sustainability/
3,«ЭЛ5-Энерго»,https://www.el5-energo.ru,https://www.el5-energo.ru/sustainability/
4,«Полиметалл»,https://www.polymetalinternational.com/ru/,https://www.polymetalinternational.com/ru/sust...
5,"«ФосАгро», группа",https://www.phosagro.ru,https://www.phosagro.ru/sustainability/
6,«МОСКОВСКИЙ КРЕДИТНЫЙ БАНК»,https://mkb.ru,https://ir.mkb.ru/sustainability
7,"«Норильский никель», горно-металлургическая ко...",https://www.nornickel.ru,https://www.nornickel.ru/sustainability/
8,«Северсталь»,https://severstal.com/rus/,https://severstal.com/rus/sustainable-developm...
9,«ЛУКОЙЛ»,https://lukoil.ru,https://lukoil.ru/Sustainability


В таблице RAEX содержатся столбцы `url` (ссылка на главную страницу сайта компании) и `url_sustainability` (ссылка на страницу с информацией об устойчивом развитии компании). Приоритетным для нас является второе, однако поле `url_sustainability` заполнено не у всех компаний.

Наша задача получить список компаний и их url. Если у компании есть `url_sustainability` и он подходящего формата, рассматриваем его. Иначе — берем url главной страницы сайта.

In [47]:
url_list = pd.DataFrame()

url_list['name'] = choose_url_list['Название']
url_list['url'] = choose_url_list['url_sustainability'].mask(choose_url_list['url_sustainability'].isnull() | np.logical_not(choose_url_list['url_sustainability'].str.startswith('http')), choose_url_list['url'])
url_list

,name,url
0,НЛМК,https://nlmk.com/ru/sustainability/
1,«Полюс»,https://sustainability.polyus.com/ru/
2,«Уралкалий»,https://www.uralkali.com/ru/sustainability/
3,«ЭЛ5-Энерго»,https://www.el5-energo.ru/sustainability/
4,«Полиметалл»,https://www.polymetalinternational.com/ru/sust...
5,"«ФосАгро», группа",https://www.phosagro.ru/sustainability/
6,«МОСКОВСКИЙ КРЕДИТНЫЙ БАНК»,https://ir.mkb.ru/sustainability
7,"«Норильский никель», горно-металлургическая ко...",https://www.nornickel.ru/sustainability/
8,«Северсталь»,https://severstal.com/rus/sustainable-developm...
9,«ЛУКОЙЛ»,https://lukoil.ru/Sustainability


# Парсер сайта компании

In [94]:
def parse_site(name, url, MAX_DEPTH = 2):
  start_link = (url, '/', 0) # url, parent_url, level

  visited_urls = set()
  queue = deque([start_link])

  while queue:
      url, parent_url, level = queue.popleft()
      if url in visited_urls:
          continue

      try:
          response = requests.get(url=url, timeout=timeout, headers=headers, verify=False)
          soup = BeautifulSoup(response.content, 'html.parser')

          # Парсинг текста со страницы
          marked_text = mark_text(response.text)
          data['id'].append(len(data['id']) + 1)
          data['name'].append(name)
          data['url'].append(url)
          data['parent_url'].append(parent_url)
          data['level'].append(level)
          data['text'].append(marked_text)

          cur.execute("INSERT OR IGNORE INTO Pages (CompanyName, PageURL, ParentPageURL, PageLevel, PageText)"
                    " VALUES (?, ?, ?, ?, ?)", (name, url, parent_url, level, marked_text))

          visited_urls.add(url)

          if level < MAX_DEPTH:
                    links = [
                        a.get('href') for a in soup.find_all('a')
                        if a.get('href') and a.get('href').startswith('/')
                    ]
                    # print(links)
                    links = [link[1:] for link in links]
                    links = [*set(links)]
                    links = list(filter(None, links))

                    for link in links:
                            if ".pdf" not in link and ".PDF" not in link:
                                next_url = urljoin(url[:url.rfind('.com')] + '.com', link)
                                if next_url not in visited_urls and next_url.startswith('http') and urlparse(next_url).netloc == urlparse(start_link[0]).netloc:
                                    queue.append((next_url, url, level + 1))
      except Exception as e:
          print(f'Error parsing {url}: {e}')

# Тестирование работы парсера на 10 веб-сайтах компаний

In [95]:
# Подключение базы данных и создание таблицы
conn = sqlite3.connect('history.db')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Pages (
    PageID INTEGER PRIMARY KEY AUTOINCREMENT,
    CompanyName TEXT,
    PageURL TEXT,
    ParentPageURL TEXT,
    PageLevel INTEGER,
    PageText TEXT)
''')

# Параллельное создание датафрейма для виуализации
data = {'id': [], 'name': [], 'url': [], 'parent_url': [], 'level': [], 'text': []}
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
    "accept": "application/json"
}

timeout = (5, 25)

NUMBER_OF_SITES_TO_PARSE = 2

i = 0
for index, row in url_list.iterrows():
  parse_site(row['name'], row['url'], 1)
  if i + 1 == NUMBER_OF_SITES_TO_PARSE:
    break
  i += 1

conn.commit()
df = pd.DataFrame(data)
print(df)

      id     name                                                url  \
0      1     НЛМК                https://nlmk.com/ru/sustainability/   
1      2     НЛМК              https://nlmk.com/ru/ir/presentations/   
2      3     НЛМК               https://nlmk.com/ru/about/key-facts/   
3      4     НЛМК       https://nlmk.com/ru/media-center/multimedia/   
4      5     НЛМК  https://nlmk.com/ru/media-center/press-release...   
5      6     НЛМК  https://nlmk.com/ru/sustainability/our-approac...   
6      7     НЛМК  https://nlmk.com/ru/about/governance/corporate...   
7      8     НЛМК     https://nlmk.com/ru/ir/results/annual-reports/   
8      9     НЛМК          https://nlmk.com/ru/ir/for-ESG-investors/   
9     10     НЛМК  https://nlmk.com/ru/sustainability/anticorrupt...   
10    11     НЛМК  https://nlmk.com/ru/sustainability/anticorrupt...   
11    12     НЛМК        https://nlmk.com/ru/ir/results/csr-reports/   
12    13     НЛМК        https://nlmk.com/ru/media-center/press-

In [97]:
# Посмотрим содержимое таблицы в базе данных

cur.execute("SELECT * FROM Pages")
rows = cur.fetchall()

for row in rows:
    print(row)

# conn.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

